# API Keys

In [5]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

# Define Tools

In [6]:
from tools import Tools

tools = Tools()
tool_dict = tools.tool_dict

for tool in tool_dict:
    print(tool)

none
website_info_retriever
website_links_crawler
website_reader
pdf_reader


# Define Agent

## Read Agent Parameter (yaml)

In [7]:
import yaml

# ! 注意yaml檔案版本
with open('agents_parameter.yaml', 'r', encoding="utf-8") as file:
    agents_parameter = yaml.safe_load(file)

## Define Planner

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List


class Plan(BaseModel):
    """Plan to follow in future"""

    steps: List[str] = Field(
        description="different steps to follow, should be in sorted order"
    )

# TODO 改成讀取yaml檔案形式
planner_model = agents_parameter["Planner"]["model"]
print("planner_model: " + planner_model)
planner_system_prompt = agents_parameter["Planner"]["prompt"]
print("planner_system_prompt: \n" + planner_system_prompt)
planner_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            planner_system_prompt,
        ),
        ("placeholder", "{messages}"), # placeholer 用來動態嵌入使用者輸入的訊息
    ]
)

planner_llm = ChatOpenAI(model=planner_model, temperature=0)
planner = planner_prompt | planner_llm.with_structured_output(Plan) # 限制使用特定模板回答問題

planner_model: gpt-4o-mini
planner_system_prompt: 
You are a planning agent in an LLM-based multi-agent system designed to assist users in navigating and understanding school websites.

Your job is to generate clear, logical, and actionable step-by-step plans that guide other agents to fulfill the user's request. Each plan step should include:
  - A brief explanation of what the step aims to accomplish
  - A clear description of what needs to be found or processed
  - An output placeholder (e.g., #E1, #E2, etc.) for use in later steps

Please strictly follow this reasoning framework when designing your plan:
  1. Begin by identifying which website(s) from a pre-constructed internal database are most relevant to the user's input or intent.
  2. Analyze the content of the selected website to determine whether it contains enough information to address the user's need.
  3. If the content is **insufficient**, locate hyperlinks or references within the page that are most likely to lead to m

In [10]:
response = planner.invoke({"messages": [("user", "Who is the headmaster of National Central University in Taiwan?")]})
for step in response.steps:
    print(step)

Search the internal website database to find the site most likely to contain information about the headmaster of National Central University in Taiwan. #E1 = [The most relevant website url based on user input]
Read the content of the selected site to understand what information it provides regarding the headmaster. #E2 = [Page content of #E1]
Evaluate whether the content in #E2 is sufficient to satisfy the user's request about the headmaster. #E3 = [Assessment of sufficiency]
If #E3 suggests the information is insufficient, identify in-page links that are most relevant to the user's input regarding the headmaster. #E4 = [List of relevant hyperlinks from #E2]
If #E3 suggests the information is insufficient, choose the most promising hyperlink and repeat the content reading step. #E5 = [Content of newly navigated page]
If the information is sufficient, extract the relevant details that directly address the user's request about the headmaster. #E6 = [Final answer based on #E2 or #E5]


## Define Executor

In [13]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults

executor_model = agents_parameter["Executor"]["model"]
print("executor_model: " + executor_model)
executor_llm = ChatOpenAI(model=executor_model)
executor_prompt = agents_parameter["Executor"]["prompt"]
print("executor_prompt: \n" + executor_prompt)

# tools = [TavilySearchResults(max_results=3)] # 測試時暫時用TavilySearchResults這個工具
executor_tool_list = [tool_dict[name] for name in agents_parameter['Executor']['tool_list']]
print("executor_tool_list: ")
for tool in executor_tool_list:
    print(tool.name)

executor = create_react_agent(executor_llm, executor_tool_list, prompt=executor_prompt)

executor_model: gpt-4o-mini
executor_prompt: 
You are a helpful assistant.

executor_tool_list: 
website_info_retriever
website_links_crawler
website_reader
pdf_reader


In [14]:
response = executor.invoke({"messages": [("user", "Who is the headmaster of National Central University in Taiwan?")]})
for message in response["messages"]:
    print(message)

link:  https://www.ncu.edu.tw
page_content:  網站標題: National Central University
簡介: 國立中央大學。
---
link:  https://www.ncu.edu.tw
page_content:  網站標題: National Central University
簡介: 國立中央大學（National Central University，簡稱NCU）是台灣的一所國立大學，成立於1962年，位於桃園市。該校以其卓越的學術研究和多元化的學科聞名，涵蓋科學、工程、管理、人文社會科學等領域。國立中央大學積極推動國際合作與交流，並致力於科技創新及社會服務，培養具全球視野和專業能力的人才。校內設施完善，擁有現代化的實驗室和圖書館，並提供多樣的課外活動和學習資源，旨在提升學生的整體素質與競爭力。
---
link:  http://www.ncu.edu.tw
page_content:  網站標題: 中央大學
簡介: 國立中央大學（National Central University, NCU）是台灣一所著名的高等教育機構，成立於1962年，位於桃園市。該校提供多樣化的學術課程，涵蓋科學、工程、文學、社會科學等領域，致力於培養具備創新能力和國際視野的人才。中央大學擁有優秀的師資和研究環境，積極進行學術研究和產學合作，並與多所國際知名大學建立合作關係。此外，該校也注重學生的全面發展，提供豐富的課外活動和資源支持。
---
link:  https://ncusec.ncu.edu.tw/preselection/
page_content:  網站標題: 校長遴選專區
簡介: 國立中央大學的第九任校長遴選專區提供公告訊息、相關法規、委員名單、表單下載及聯絡資訊。根據最新公告，蕭述三教授已獲教育部同意聘任為校長，並於2024年10月30日正式上任。遴選過程中，包括候選人治校理念說明會和投票結果的相關會議紀錄均已公開。聯絡人為陳敏茲小姐及吳秋萍組長，聯絡電話及校址也已提供。
---
link:  https://www.ncu.edu.tw/
page_content:  網站標題: 中大首頁
簡介: 國立中央大學（National Central University, NCU）位於台灣，是一所

## Define Replanner

In [ ]:
from typing import Union


class Response(BaseModel):
    """Response to user."""

    response: str


class Act(BaseModel):
    """Action to perform."""

    action: Union[Response, Plan] = Field(
        description="Action to perform. If you want to respond to user, use Response. "
        "If you need to further use tools to get the answer, use Plan."
    )


replanner_prompt = ChatPromptTemplate.from_template(
    """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

Your objective was this:
{input}

Your original plan was this:
{plan}

You have currently done the follow steps:
{past_steps}

Update your plan accordingly. If no more steps are needed and you can return to the user, then respond with that. Otherwise, fill out the plan. Only add steps to the plan that still NEED to be done. Do not return previously done steps as part of the plan."""
)

replanner_llm = ChatOpenAI(model="gpt-4o", temperature=0) # ! Replanner需要使用gpt-4o才不會一直call tools
replanner = replanner_prompt | replanner_llm.with_structured_output(Act) # 限制使用特定模板回答問題

# Define Graph State

In [15]:
import operator
from typing import Annotated, List, Tuple
from typing_extensions import TypedDict


class PlanExecute(TypedDict):
    input: str
    plan: List[str]
    past_steps: Annotated[List[Tuple], operator.add]
    response: str

# Create the Graph

In [ ]:
from typing import Literal
from langgraph.graph import END


async def execute_step(state: PlanExecute):
    plan = state["plan"]
    plan_str = "\n".join(f"{i+1}. {step}" for i, step in enumerate(plan))
    task = plan[0]
    task_formatted = f"""For the following plan:
{plan_str}\n\nYou are tasked with executing step {1}, {task}."""
    agent_response = await executor.ainvoke(
        {"messages": [("user", task_formatted)]}
    )
    return {
        "past_steps": [(task, agent_response["messages"][-1].content)],
    }


async def plan_step(state: PlanExecute):
    plan = await planner.ainvoke({"messages": [("user", state["input"])]}) # 對應到planner system prompt中的{messages}
    return {"plan": plan.steps}


async def replan_step(state: PlanExecute):
    output = await replanner.ainvoke(state)
    if isinstance(output.action, Response):
        return {"response": output.action.response}
    else:
        return {"plan": output.action.steps}


def should_end(state: PlanExecute):
    if "response" in state and state["response"]:
        return END
    else:
        return "agent"

In [ ]:
from langgraph.graph import StateGraph, START

workflow = StateGraph(PlanExecute)

# Add the plan node
workflow.add_node("planner", plan_step)

# Add the execution step
workflow.add_node("agent", execute_step)

# Add a replan node
workflow.add_node("replan", replan_step)

workflow.add_edge(START, "planner")

# From plan we go to agent
workflow.add_edge("planner", "agent")

# From agent, we replan
workflow.add_edge("agent", "replan")

workflow.add_conditional_edges(
    "replan",
    # Next, we pass in the function that will determine which node is called next.
    should_end,
    ["agent", END],
)

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [ ]:
from IPython.display import Image, display

display(Image(app.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
# Who is the headmaster of National Central University in Taiwan?
config = {"recursion_limit": 50}
inputs = {"input": "Who is the headmaster of National Central University in Taiwan?"}
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(v)